In [63]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

with open("keys.txt", "r") as f:
    keys = f.readlines()

key, secret_key = [x.strip() for x in keys]
trading_client = TradingClient(key, secret_key, paper=True)

## Get data from Alpaca

In [95]:
account = trading_client.get_account()
positions = trading_client.get_all_positions()
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)
assets = trading_client.get_all_assets(search_params)

equity = float(account.equity)
if len(positions) > 0:
    positions = {x.symbol: int(x.qty) for x in positions}
else:
    positions = {}
tradable = [x.symbol for x in assets if x.tradable]
shortable = [x.symbol for x in assets if x.shortable]

## Build Feature Dataset

- Don't need much history.  Start here in 2022.
- And don't need weekly returns (after computing momentum).

In [69]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
driver = 'SQL+Server'
string = f"mssql+pyodbc://{username}:{password}@{server}/{database}" 
try: 
    conn = create_engine(string + "?driver='SQL+Server'").connect()
except:
    try:
        conn = create_engine(string + "?driver='ODBC+Driver+18+for+SQL+Server'").connect()
    except:
        import pymssql
        string = f"mssql+pymssql://{username}:{password}@{server}/{database}"   
        conn = create_engine(string).connect() 

In [70]:
sep_weekly = pd.read_sql(
    """ 
    select date, ticker, closeadj, closeunadj, volume, lastupdated from sep_weekly 
    where date >= '2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
sep_weekly = sep_weekly.groupby(["ticker", "date"]).last()
sep_weekly = sep_weekly.drop(columns=["lastupdated"])

ret = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change()
ret.name = "ret"

price = sep_weekly.closeunadj
price.name = "price"

volume = sep_weekly.volume 
volume.name = "volume"

In [71]:
ret_annual = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(52)
ret_monthly = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(4)
mom = (1 + ret_annual) / (1 + ret_monthly) - 1
mom.name = "mom"

In [72]:
weekly = pd.read_sql(
    """ 
    select date, ticker, pb, marketcap, lastupdated from weekly 
    where date>='2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
weekly = weekly.groupby(["ticker", "date"]).last()
weekly = weekly.drop(columns=["lastupdated"])

pb = weekly.pb
pb.name = "pb" 
marketcap = weekly.marketcap 
marketcap.name = "marketcap"

In [73]:
sf1 = pd.read_sql(
    """ 
    select datekey as date, ticker, assets, netinc, equity, lastupdated from sf1
    where datekey>='2022-01-01' and dimension='ARY' and assets>0 and equity>0
    order by ticker, datekey, lastupdated    
    """,
    conn,
)
sf1 = sf1.groupby(["ticker", "date"]).last()
sf1 = sf1.drop(columns=["lastupdated"])

# change dates to Fridays
from datetime import timedelta 
sf1 = sf1.reset_index()
sf1.date =sf1.date.map(
    lambda x: x + timedelta(4 - x.weekday())
)
sf1 = sf1.set_index(["ticker", "date"])
sf1 = sf1[~sf1.index.duplicated()]

assets = sf1.assets
assets.name = "assets" 
netinc = sf1.netinc 
netinc.name = "netinc" 
equity = sf1.equity
equity.name = "equity"

equity = equity.groupby("ticker", group_keys=False).shift() 
roe = netinc / equity 
roe.name = "roe"

assetgr = assets.groupby("ticker", group_keys=False).pct_change()
assetgr.name = "assetgr"

In [74]:
df = pd.concat(
    (
        mom, 
        volume,
        price, 
        pb, 
        marketcap, 
        roe, 
        assetgr
        ), 
        axis=1
    )
df["roe"] = df.groupby("ticker", group_keys=False).roe.ffill()
df["assetgr"] = df.groupby("ticker", group_keys=False).assetgr.ffill()

df = df.reset_index()
df.date = df.date.astype(str)
df = df[df.date==df.date.max()]
df = df[df.price >= 5]
df = df.dropna()

features = [
    "mom",
    "volume",
    "pb",
    "marketcap",
    "roe",
    "assetgr" 
]

In [75]:
industries = pd.read_sql(
    """ 
    select ticker, famaindustry as industry from tickers   
    """,
    conn,
)
industries["industry"] = industries.industry.fillna("Almost Nothing")
df = df.merge(industries, on="ticker", how="left")
df = df.dropna()

In [76]:
for x in features:
    df[f"{x}_industry"] = df.groupby(
        ["industry"], 
        group_keys=False
    )[x].apply(
        lambda x: x - x.median()
    )

features += [f"{x}_industry" for x in features]

In [77]:
for f in features:
    df[f] = df[f].rank(pct=True)

## Load Model and Predict

In [78]:
from joblib import load
model = load("mymodel.joblib")
df["predict"] = model.predict(df[features])

### Best and worst stocks

In [114]:
df = df.sort_values(by="predict", ascending=False)
best = df[["ticker", "predict"]].copy().reset_index(drop=True)
best = best[best.ticker.isin(tradable)].iloc[:50]

df = df.sort_values(by="predict", ascending=True)
worst = df[["ticker", "predict"]].copy().reset_index(drop=True)
worst = worst[worst.ticker.isin(shortable)].iloc[:50]

In [115]:
best

ticker    predict
0    SMCI  51.394069
1     QSR  51.306917
2    MPWR  51.278958
3    ODFL  51.278958
4    FERG  51.278958
5    FAST  51.272102
6      TT  51.268591
7    SNPS  51.268591
8    SCCO  51.268591
9    LULU  51.268591
10    WST  51.268043
11    CCI  51.266315
12   MCHP  51.265236
13      A  51.265236
14   VEEV  51.265101
15     BR  51.264289
16    EFX  51.263788
17    ACN  51.261735
18    MMC  51.261735
19     WM  51.261735
20   ROST  51.261735
21    APH  51.261735
22    UNP  51.261735
23     BX  51.261735
24    CAT  51.261735
25   NFLX  51.261735
26   CPRT  51.261735
27   COST  51.261735
28   ANET  51.261735
29     PG  51.261735
30    BAH  51.260328
31    AVY  51.260328
32   NDAQ  51.258245
33    IQV  51.258001
34    AMP  51.255487
35   CTAS  51.255270
36    CMG  51.255270
37    AXP  51.254869
38    UNH  51.254869
39     EW  51.254869
40   TSCO  51.254139
41   VRSK  51.254100
42   CDNS  51.253883
43    JBL  51.253462
44    MOH  51.253462
45     PH  51.253421
46    NSC  51.253421
47   KEYS  51.252873
48    TJX  51.251585
49   ADBE  51.251585

In [116]:
worst

ticker    predict
3    EIGR  39.167009
8    PRPO  42.381558
14   BODY  44.111180
16   KPLT  44.205348
18   CALC  44.351845
19    XOS  44.567180
22   AIRT  44.969343
23   ONCT  45.006040
24   ECOR  45.024591
25   SKLZ  45.068362
29   AEYE  45.264590
32   NVNO  45.458468
33   IPWR  45.482758
34    LEE  45.507330
37   CRVO  45.558953
39    TSE  45.651877
40   CRIS  45.681653
45   GANX  45.987356
46   RVNC  46.003517
47   NGNE  46.135566
49   TELA  46.167795
50   OPAD  46.228000
52   SYRE  46.358609
53   ATNM  46.371394
56    TIL  46.443035
57    VHC  46.446717
58   SILK  46.475485
59   ARQT  46.515012
61   ROOT  46.533247
62   TSVT  46.542394
66   ENTA  46.682079
68    DLA  46.756579
69   CULP  46.757973
70   GOCO  46.763119
71   PRAX  46.789427
72   SCOR  46.802425
73   QUIK  46.824635
74   SYRS  46.829105
77   CSBR  46.903732
80   CELC  47.048236
81    AIP  47.050116
83   IMRX  47.097642
84   AOMR  47.099600
85   AKYA  47.108796
86    BIG  47.125909
88   ABEO  47.136489
89   CXDO  47.190025
90   HIPO  47.205715
91   OUST  47.218883
92   CLPT  47.223387

### Close unwanted positions

In [110]:
positions_to_close = [
    x for x in positions 
    if (x not in best.ticker.to_list())
    and (x not in worst.ticker.to_list())
]
for x in positions_to_close:
    current = positions[x]
    trading_client.submit_order(
        order_data=MarketOrderRequest(
            symbol=x,
            qty=abs(current),
            side=OrderSide.BUY if current<0 else OrderSide.SELL,
            time_in_force=TimeInForce.DAY
        )
    )

APIError: {"code":40310000,"existing_order_id":"24608f70-d4e2-4473-b57f-89ae26b68096","message":"potential wash trade detected. use complex orders","reject_reason":"opposite side market/stop order exists"}

In [107]:
abs(-2)

2

In [103]:
positions.keys()

dict_keys(['AAPL', 'SPY'])

### Account equity

### Positions

#### Tradable and shortable 

## Trade

### Rebalance SPY

In [ ]:
import yfinance as yf 
price = yf.download("SPY", start=2024)["Close"].iloc[-1]
shares = int(equity / price)

market_order_data = MarketOrderRequest(
    symbol=tick,
    qty=-df.loc[tick, "trade"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )

#### Calculate target positions

In [ ]:
numstocks = 50 ## number long and number short 
dollars_per_stock = 0.4*equity / numstocks


df = df.sort_values(by="rnk")

try:
    short_cutoff = df[df.shortable & (df.bid>0)].rnk.iloc[numstocks-1]
    long_cutoff = df[df.tradable & (df.ask>0)].rnk.iloc[-numstocks]
    df["target"] = np.where(
        df.shortable & (df.bid>0) & (df.rnk<=short_cutoff),
        -short_per_stock/df.bid, 
        0
    )
    df["target"] = np.where(
        df.tradable & (df.ask>0) & (df.rnk>=long_cutoff), 
        long_per_stock/df.ask, 
        df.target
    )
    df["target"] = df.target.astype(int)
except:
    df["target"] = 0

#### Calculate trades

Using a simple but suboptimal protocol: trade to target positions without trying to minimize the number of round trips we might eventually make.

In [ ]:
df["trade"] = df.target - df.current

#### Make trades

In [ ]:
for tick in df.index: 
    if df.loc[tick, "trade"]<0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=-df.loc[tick, "trade"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"sell order for {tick} failed")
    elif df.loc[tick, "trade"]>0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=df.loc[tick, "trade"],
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"buy order for {tick} failed")



## Save data

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")

df["date"] = today

try:
    d = pd.read_csv("files/trade_data.csv", index_col="ticker")
    d = d[d.date != today]
    df = pd.concat((d, df))
    df.to_csv("files/trade_data.csv")
except:
    df.to_csv("files/trade_data.csv")

In [ ]:
account = trading_client.get_account()
account = dict(account)
account = pd.DataFrame(pd.Series(account)).T
account["date"] = today

try:
    d = pd.read_csv("files/account.csv")
    d = d[d.date != today]
    account = pd.concat((d, account))
    account.to_csv("files/account.csv")
except:
    account.to_csv("files/account.csv")


C:\Users\keb7\AppData\Local\Temp\ipykernel_107440\1684087778.py:9: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  account = pd.concat((d, account))


In [ ]:
positions = trading_client.get_all_positions()
positions = {x.symbol: x.qty for x in positions}
positions = pd.DataFrame(pd.Series(positions))
positions["date"] = today

try:
    d = pd.read_csv("files/positions.csv")
    d = d[d.date != today]
    positions = pd.concat((d, positions))
    positions.to_csv("files/positions.csv")
except:
    positions.to_csv("files/positions.csv")